In [1]:
# Carlos Eduardo Velasco Elenes
# A01708634
# November 10th 2023

# Multiple agents grid package
from mesa import Agent, Model 

# Multiple agents in one cell package
from mesa.space import MultiGrid

# Instantiate agents at the same time package
from mesa.time import RandomActivation

# Numeric values management
import numpy as np

# Time measurement package
import time
# Random package
import random

In [2]:
class RobotAgent(Agent):
    # Initialize de agent
    def __init__(self, id, model):
        super().__init__(id, model)
        
    # Next decision of the agent
    def step(self):
        # Obtain the current position of the agent
        x, y = self.pos

        # If the current position is cleaned, then move
        if self.model.is_clean(x, y):
            # Move the agent
            self.move()

        # Else, clean the cell and then move
        else:
            # We call the clean_cell method from the model and we pass the coordinates of the position
            self.model.clean_cell(x, y)
            # Move the agent
            self.move()

    # Move the agent
    def move(self):
        # Gets all the neighbours
        possible_moves = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        # Calculates which cells are available
        possible_moves = [cell for cell in possible_moves if self.model.grid.is_cell_empty(cell)]

        # If there are cells available, then move
        if possible_moves:
            # Chooses a random cell to move
            new_position = random.choice(possible_moves)
            # Move the agent
            self.model.grid.move_agent(self, new_position)


In [3]:
class RobotModel(Model):
    def __init__(self, width, height, num_agents, dirty_cells):
        # Create the grid (since we will be positioning the agents in the same position, we need to have a multigrid)
        self.grid = MultiGrid(width, height, torus = False)
        # Activate all of the agents at the same time
        self.schedule = RandomActivation(self)
        # Attribute to store the values of the cells in the grid
        self.dirty = np.zeros((width, height))
        # Attribute to store the amount of dirty cells
        self.total_dirty_cells = 1
        # Attribute to count the amount of cells cleaned
        self.cleaned_cells = 0
        # Attribute to count the amount of steps the simulation is taking
        self.steps = 0

        id = 0

        # Instantiate the agents in the position (1,1). The number of agents that will be instantiated are specified by the parameter num_agents
        for _ in range(num_agents):
            # Initialize the agent
            agent = RobotAgent(id, self)
            # Add it to the schedule
            self.schedule.add(agent)
            # Place the agent in (1, 1)
            self.grid.place_agent(agent, (1, 1))
            # Increment the id to guarantee unique keys
            id += 1


        # Total cells available
        total_cells = width * height
        # Number of dirty cells
        self.total_dirty_cells = int(dirty_cells * total_cells)
        # Place the dirty cells in the grid at random positions
        for _ in range(self.total_dirty_cells):
            # Get the random coordinates for the new dirty cell
            x, y = self.random_empty_cell()
            # Set the value of that cell to 1
            self.dirty[x][y] = 1

    # Continue with the next step 
    def step(self):
        self.schedule.step()
        self.steps += 1

    # Method to clean the dirty cell 
    def clean_cell(self, x, y):
        # Change the value to zero
        self.dirty[x][y] = 0
        # Increment the amount of cleaned cells
        self.cleaned_cells += 1

    # Method to check if the cell's cleaned (value = 0) or dirty (value = 1)
    def is_clean(self, x, y):
        return self.dirty[x][y] == 0
    
    # Method to calculate the percentage of dirty cells still left
    def calculate_dirty_cells(self):
        return round(100 - (self.cleaned_cells / self.total_dirty_cells) * 100, 2)
    
    # Method to obtain the amount of steps required for the simulation
    def get_steps(self):
        return self.steps
    
    # Method to get the total amount of dirty cells
    def get_dirty_cells(self):
        return self.total_dirty_cells
    
    # Method to get the total amount of cleaned cells
    def get_cleaned_cells(self):
        return self.cleaned_cells

    # Method to obtain the coordinates of a random cell that is available
    def random_empty_cell(self):
        # Get a random empty cell in the grid
        x, y = self.random_pos()
        # If that cell is not available, then find another cell
        while not self.grid.is_cell_empty((x, y)) or self.dirty[x][y] == 1:
            x, y = self.random_pos()
        return x, y
    
    # Method to select a random cell from the grid
    def random_pos(self):
        return self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)

In [4]:
# Constants definition
WIDTH = 100
HEIGHT = 100
DIRTY_CELLS = 0.9
MAX_STEPS = 500000
NUM_AGENTS = 1

# Start the timer
start_time = round(time.time(), 2)
# Create the model
model = RobotModel(WIDTH, HEIGHT, NUM_AGENTS, DIRTY_CELLS)
# Print the initial percentage and amount of dirty cells
print(f"The initial percentage of cells to be cleaned is: {model.calculate_dirty_cells()}% ({model.get_cleaned_cells()}/{model.get_dirty_cells()} cells cleaned)")
# Iteration of the model defined by the maximum amount of steps
for i in range(MAX_STEPS):
    # If there are no more cells to be cleaned, then exit the loop
    if model.calculate_dirty_cells() == 0.0:
        break
    model.step()
# Stop the timer
end_time = round(time.time() - start_time, 2)

# Prints the percentage of cells left to be cleaned and how many of the dirty cells were cleaned
print(f"The percentage of cells to be cleaned left is: {model.calculate_dirty_cells()}% ({model.get_cleaned_cells()}/{model.get_dirty_cells()} cells cleaned)")
# Prints the time required to finish the simulation
print(f"Time required to finish the simulation: {end_time} seconds")
# Prints the number of steps required for the simulation
print(f"Number of steps required for the simulation: {model.get_steps()} steps")

The initial percentage of cells to be cleaned is: 100.0% (0/9000 cells cleaned)
The percentage of cells to be cleaned left is: 0.0% (9000/9000 cells cleaned)
Time required to finish the simulation: 7.43 seconds
Number of steps required for the simulation: 226743 steps
